### INTRO

* Basis for manifolds, spectral clustering, more
* classification (discrete labels), regression (continuous labels)
* Principle:
   * Find predefined #samples closest to new point - predict label
   * #samples: user-defined (kNN) or local density (radius based)
* #samples possibly transformed into fast index struct [ball tree](http://scikit-learn.org/stable/modules/neighbors.html#ball-tree) or [KD tree](http://scikit-learn.org/stable/modules/neighbors.html#kd-tree)
   
[class API](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.neighbors)

### UNSUPERVISED NN

[API](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)

* search options: `algorithm`='auto','ball_tree','kd_tree','brute'

In [3]:
# finding nearest neighbors between 2 datasets

from sklearn.neighbors import NearestNeighbors
import numpy as np

X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)

distances, indices = nbrs.kneighbors(X)

In [4]:
indices

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]])

In [5]:
distances

array([[ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.41421356],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.41421356]])

In [6]:
# build sparse graph showing connections betw neighboring points

nbrs.kneighbors_graph(X).toarray()

array([[ 1.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  1.]])

In [1]:
# KDTree
from sklearn.neighbors import KDTree
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
kdt = KDTree(X, leaf_size=30, metric='euclidean')
kdt.query(X, k=2, return_distance=False)

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]])

In [2]:
# BallTree
from sklearn.neighbors import BallTree
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
kdt = BallTree(X, leaf_size=30, metric='euclidean')
kdt.query(X, k=2, return_distance=False)

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]])

### NN CLASSIFICATION

[API by KNN](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier) |
[API by radius](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsClassifier.html#sklearn.neighbors.RadiusNeighborsClassifier)

* instance-based learning (doesn't build model, just stores training instances)
* decision function = simple majority vote of NNs of each point
* KNN more popular
* Consider radius-basis when data not uniformly sampled
* Radius function less effective in highD use cases
* distance weightings can be set with `weights`='uniform'|'distance'

[NN, iris dataset](plot_classification.ipynb)

In [3]:
X = [[0], [1], [2], [3]]
y = [0, 0, 1, 1]
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y) 
print(neigh.predict([[1.1]]))
print(neigh.predict_proba([[0.9]]))

[0]
[[ 0.66666667  0.33333333]]


In [4]:
# what point is closest to [1,1,1]?

samples = [[0., 0., 0.], [0., .5, 0.], [1., 1., .5]]
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(samples) 

print(neigh.kneighbors([[1., 1., 1.]])) 

# answer: element distance = 0.5, is the 3rd element in sample array

(array([[ 0.5]]), array([[2]]))


In [5]:
# multiple point query
X = [[0., 1., 0.], [1., 0., 1.]]
neigh.kneighbors(X, return_distance=False) 

array([[1],
       [2]])

In [6]:
# neighbor graph:
X = [[0], [3], [1]]
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=2)
neigh.fit(X) 

A = neigh.kneighbors_graph(X)
A.toarray()

array([[ 1.,  0.,  1.],
       [ 0.,  1.,  1.],
       [ 1.,  0.,  1.]])

### NN REGRESSION

[API by KNN)](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor) | [API by radius](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsRegressor.html#sklearn.neighbors.RadiusNeighborsRegressor)

[demo](plot_regression.ipynb) | [
face completion, multi-output](plot_multioutput_face_completion.ipynb)

### ALGORITHMS

* BruteForce: scales as O(D x N^2) -- quickly becomes inefficient
* [K-D tree](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree): encodes aggregate distance, uses Cartesian axes as separators. complexity = O(D x N x log(N)). Less efficient as D increases.
* [Ball tree](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html#sklearn.neighbors.BallTree): data partitioned in nesting hyperspheres w/ centroid & radius.

*** Tradeoffs: ***

* #samples
* dimensionality (#features)
* data structure (dimensionality, sparsity)
* #neighbors requested (K)
* #query points

*** Leaf Size: ***

* brute force > tree-based for small sample sizes
* larger leaf size ==> faster tree construction (fewer nodes)
* good compromise: leaf_size = 30 to optimize search time
* as leaf_size goes up, memory rqmnts go up.

### NEAREST CENTROID

* represents each class by centroid of its membership

### NEAREST CENTROID, SHUNKEN

* uses `shrink_threshold` param
* useful for removing noisy features

[API](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid) |
[example](plot_nearest_centroid.ipynb)

In [7]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])
clf = NearestCentroid()
clf.fit(X, y)

print(clf.predict([[-0.8, -1]]))

[1]


### APPROXIMATE NEAREST NEIGHBORS

*** For apps with K>50, AKA The 'Curse of Dimensionality' ***

* When answers don't have to be exact

[LSH Forest (API)](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.LSHForest.html#sklearn.neighbors.LSHForest) |
[demo: accuracy vs #candidates,#estimators](plot_approximate_nearest_neighbors_hyperparameters.ipynb) |
[demo: query time vs #samples](plot_approximate_nearest_neighbors_scalability.ipynb)